# Lab 7 - Textual Data Analytics
Complete the code with TODO tag.
## 1. Feature Engineering
In this exercise we will understand the functioning of TF/IDF ranking. Implement the feature engineering and its application, based on the code framework provided below.

First we use textual data from Twitter.

In [1]:
import numpy as np
import pandas as pd
import math
import re
from scipy import spatial
from operator import itemgetter

data = pd.read_csv('elonmusk_tweets.csv')
print(len(data))
data.head()

2819


,id,created_at,text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


### 1.1. Text Normalization
Now we need to normalize text by stemming, tokenizing, and removing stopwords.

In [2]:
from __future__ import print_function, division
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
nltk.download('punkt')
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import pprint 
pp = pprint.PrettyPrinter(indent=4)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import re

emotes = r"""
    (?:
        [:=;]
        [oO\-]?
        [D\)\]\(\]/\\OpP]
    )"""

regex_str = [emotes,
            r'<[^>]+>', # HTML tags
            r'(?:@[\w_]+)', # @-mentions
            r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
            r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs

            r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
            r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
            r'(?:[\w_]+)', # other words
            r'(?:\S)' # anything else        
            ]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emotes+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(a):
    return tokens_re.findall(a)

def normalize(x):
    new_result = []
    result = tokenize(x.lower())
    for idx, word in enumerate(result, start=1):
        if len(word) > 1:
            new_result.append(word)         
    return new_result


original_documents = [x.strip() for x in data['text']] 
documents = [normalize(x) for x in original_documents]

As you can see that the normalization is still not perfect. Please feel free to improve upon (OPTIONAL), e.g. https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/

### 1.2. Implement TF-IDF
Now you need to implement TF-IDF, including creating the vocabulary, computing term frequency, and normalizing by tf-idf weights.

In [4]:
# Flatten all the documents
flat_list = [word for doc in documents for word in doc]

# TODO: remove stop words from the vocabulary
words = [word for word in flat_list if not word in stopwords.words('english')]

# TODO: we take the 500 most common words only
counts = Counter(words)
counts = sorted(counts.items(), key=lambda x: x[1])
vocabulary = counts[-500:]
print([x for x in vocabulary if x[0] == 'tesla'])
vocabulary = [x[0] for x in vocabulary]
assert len(vocabulary) == 500

# vocabulary.sort()
vocabulary[:5]

[('tesla', 272)]


['x92', 'europe', 'latest', 'yup', 'hit']

In [5]:
def tf(vocabulary, documents):
    matrix = [0] * len(documents)
    for i, document in enumerate(documents):
        counts = Counter(document)
        matrix[i] = [0] * len(vocabulary)
        for j, term in enumerate(vocabulary):
            matrix[i][j] = counts[term]
    return matrix

tf = tf(vocabulary, documents)
np.array(vocabulary)[np.where(np.array(tf[1]) > 0)], np.array(tf[1])[np.where(np.array(tf[1]) > 0)]

(array(['exactly', "that's", 'xa6', 'tesla', 'x80', 'xe2'], dtype='<U16'),
 array([1, 1, 1, 1, 1, 1]))

In [6]:
def idf(vocabulary, documents):
    """TODO: compute IDF, storing values in a dictionary"""
    idf = dict()
    C = len(documents)
    
    for word in vocabulary:
        Ct = 0
        for document in documents:
            if word in document:
                Ct += 1
        idf[word] = math.log(C/Ct)
    
    return idf

idf = idf(vocabulary, documents)
print(idf)

{'x92': 5.641552398120067, 'europe': 5.641552398120067, 'latest': 5.641552398120067, 'yup': 5.641552398120067, 'hit': 5.746912913777893, 'orders': 5.641552398120067, 'cargo': 5.641552398120067, 'put': 5.641552398120067, 'line': 5.641552398120067, 'consumer': 5.641552398120067, 'machine': 5.746912913777893, 'cause': 5.641552398120067, 'built': 5.641552398120067, 'since': 5.641552398120067, 'least': 5.641552398120067, 'planet': 5.641552398120067, 'house': 5.641552398120067, 'favorite': 5.641552398120067, 'level': 5.641552398120067, 'beyond': 5.641552398120067, "b'spacex": 5.641552398120067, 'case': 5.641552398120067, 'fuel': 5.641552398120067, "b'looks": 5.641552398120067, 'tested': 5.641552398120067, 'hw2': 5.746912913777893, 'half': 5.641552398120067, 'meeting': 5.641552398120067, 'went': 5.641552398120067, 'competition': 5.641552398120067, 'reading': 5.746912913777893, 'near': 5.641552398120067, '@theeconomist': 5.641552398120067, 'ny': 5.746912913777893, 'hold': 5.641552398120067, 'x

In [7]:
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]
    return vector

document_vectors = [vectorize(s, vocabulary, idf) for s in documents]
np.array(vocabulary)[np.where(np.array(document_vectors[1]) > 0)], np.array(document_vectors[1])[np.where(np.array(document_vectors[1]) > 0)]

(array(['exactly', "that's", 'xa6', 'tesla', 'x80', 'xe2'], dtype='<U16'),
 array([4.61193298, 4.36061855, 3.03886271, 2.35688883, 2.57349946,
        2.53249144]))

### 1.3. Compare the results with the reference implementation of scikit-learn library.

Now we use the scikit-learn library. As you can see that, the way we do text normalization affects the result. Feel free to further improve upon (OPTIONAL), e.g. https://stackoverflow.com/questions/36182502/add-stemming-support-to-countvectorizer-sklearn

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english', max_features=500)

features = tfidf.fit(original_documents)
corpus_tf_idf = tfidf.transform(original_documents) 

sum_words = corpus_tf_idf.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in tfidf.vocabulary_.items()]
print(sorted(words_freq, key = lambda x: x[1], reverse=True)[:5])
print('testla', corpus_tf_idf[1, features.vocabulary_['tesla']])

[('http', 163.54366542841234), ('https', 151.85039944652075), ('rt', 112.61998731390989), ('tesla', 95.96401470715628), ('xe2', 88.20944486346477)]
testla 0.3495243100660956


### 1.4.  Apply TF-IDF for information retrieval
We can use the vector representation of documents to implement an information retrieval system. We test with the query $Q$ = "tesla nasa"

In [9]:
def cosine_similarity(v1,v2):
    """TODO: compute cosine similarity"""
    return 1 - spatial.distance.cosine(v1, v2)

def search_vec(query, k, vocabulary, stemmer, document_vectors, original_documents):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    
    # TODO: rank the documents by cosine similarity
    scores = list()
    for i, dv in enumerate(document_vectors, start=1):
        scores.append([cosine_similarity(query_vector, dv), i])
    scores = sorted(scores, key=itemgetter(0))
    scores[::-1]
    print('Top-{0} documents'.format(k))
    for i in range(k):
        print(i, original_documents[scores[i][1]])

query = "tesla nasa"

stemmer = PorterStemmer()
vocabulary = [x.lower() for x in vocabulary]
search_vec(query, 5, vocabulary, stemmer, document_vectors, original_documents)

Top-5 documents
0 b"@ForIn2020 @waltmossberg @mims @defcon_5 Exactly. Tesla is absurdly overvalued if based on the past, but that's irr\xe2\x80\xa6 https://t.co/qQcTqkzgMl"
1 b'Stormy weather in Shortville ...'
2 b"@DaveLeeBBC @verge Coal is dying due to nat gas fracking. It's basically dead."
3 b"@Lexxxzis It's just a helicopter in helicopter's clothing"
4 b'Technology breakthrough: turns out chemtrails are actually a message from time-traveling aliens describing the secret of teleportation'


C:\Users\User\Anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


We can also use the scikit-learn library to do the retrieval.

In [10]:
new_features = tfidf.transform([query])

cosine_similarities = linear_kernel(new_features, corpus_tf_idf).flatten()
related_docs_indices = cosine_similarities.argsort()[::-1]

topk = 5
print('Top-{0} documents'.format(topk))
for i in range(topk):
    print(i, original_documents[related_docs_indices[i]])

Top-5 documents
0 b'@ashwin7002 @NASA @faa @AFPAA We have not ruled that out.'
1 b"SpaceX could not do this without NASA. Can't express enough appreciation. https://t.co/uQpI60zAV7"
2 b'@NASA launched a rocket into the northern lights http://t.co/tR2cSeMV'
3 b'Whatever happens today, we could not have done it without @NASA, but errors are ours alone and me most of all.'
4 b'RT @NASA: Updated @SpaceX #Dragon #ISS rendezvous times: NASA TV coverage begins Sunday at 3:30amET: http://t.co/qrm0Dz4jPE. Grapple at  ...'
